In [1]:
!curl -fsSL https://ollama.com/install.sh | sh 

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%   12.4%################                                    53.2%                59.6%
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
## Run `ollama serve` in one terminal
## Run `ollama run llama3.1:8b-instruct-fp16`
## Run `ollama run gemma2:9b-instruct-fp16`

In [3]:
!pip install OpenAI datasets rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.3/327.3 kB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0


In [4]:
from rich import print
import os
from openai import OpenAI
from datasets import Dataset, DatasetDict, load_dataset
import json

In [5]:
modelName = "llama3.1:8b-instruct-fp16"

In [15]:
topic = "Science"
fileName = f'synthetic_data-{topic}.jsonl'.format(topic)
print(fileName)
n_subtopics = 14
n_questions = 2

client = OpenAI(
   base_url = 'http://localhost:11434/v1', ## using ollama local endpoint
    api_key='ollama', # required, but unused
)

synthetic_data-Science.jsonl

In [16]:
# 1. Subtopics Generation

TOPIC_GENERATION_PROMPT_TEMPLATE = """\
Given a topic, generate a list of {n_subtopics} topics make it simple and subtopics in Arabic that are formally related to the topic.

The topic is: {topic}

Only provide the list of subtopics, without numbers, descriptions, or any additional text. Separate each subtopic with newline
"""

def generate_subtopics(client, topic, n_subtopics):
    prompt = TOPIC_GENERATION_PROMPT_TEMPLATE.format(topic=topic, n_subtopics=n_subtopics)
    response = client.chat.completions.create(
        model=modelName,
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    return response

responses = generate_subtopics(client, topic=topic, n_subtopics=n_subtopics)
print(responses.choices[0].message.content.split('\n'), len(responses.choices[0].message.content.split('\n')))

[
    'علم الفلك',
    'الجزيئات والكيمياء',
    '',
    'الفيزياء',
    'الكيمياء الحيوية',
    '',
    'التكنولوجيا',
    'البيئة والمناخ',
    '',
    'الرياضيات',
    'الجيولوجيا',
    '',
    'الأحياء',
    'الطب'
]
14

In [18]:

# 2. Questions Generation

QUESTION_PROMPT_TEMPLATE = """
Given a topic, generate {n_questions} in-depth and questions in Arabic that could be asked about that topic.

The topic is: {sub_topic}

Only provide the list of questions, formatted with each question on a new line. Exclude numbering, bullet points, and any additional text.
"""
subtopic_list = responses.choices[0].message.content.split("\n")
def generate_questions(client, sub_topic, n_questions):
    prompt = QUESTION_PROMPT_TEMPLATE.format(sub_topic=sub_topic, n_questions=n_questions)
    response = client.chat.completions.create(
        model=modelName,
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=8048,
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content

def question_generator(client, subtopic_list, n_question):
    tasks = []
    for subtopic in subtopic_list:
        tasks.append(generate_questions(client, subtopic, n_question))

    print(tasks)
    question_list = tasks
    return question_list

question_list = question_generator(client, subtopic_list, n_questions)

question_list_formatted = []
for question_set in question_list:
    question_list_formatted.extend([question.strip() for question in question_set.split("\n") if question])

ما هي أهمية علم الفلك في فهم تشكل الكون وتطوره؟
كيف يمكن استخدام تقنيات الفضاء الحديثة لتحسين معرفتنا بظواهر الفلك والنجوم؟

ما هي خصائص الجزيئات في الكيمياء؟

كم عدد أنواع الجزيئات الموجودة في الطبيعة؟

ما هي أهمية تطبيق مبادئ التنمية المستدامة في المجتمعات الحضرية؟ 
كيف يمكن أن يؤثر نمو السكان على الموارد الطبيعية والبيئة في المناطق الحضرية؟

ما هي العلاقة بين القوة والجاذبية فيزيائياً؟

هل يمكن أن تؤثر الظروف البيئية على قياس المادة في الفيزياء؟

ما هي العوامل التي تؤثر على توازن المكونات في الخلايا الحية؟
كيف يتفاعل الجينوم مع البيئة الخارجية لتحقيق التوازن البيولوجي؟

...

ما هي التغيرات التي طرأت على سطح الأرض منذ العصر الحجري؟

كيف تؤثر العمليات الجيولوجية على البيئة المحيطة بالمناطق الساحلية؟

ما هي أهمية التوعية بالصحة النفسية في المجتمع؟
كيف يمكن أن تؤثر العوامل البيئية على الصحة النفسية للفرد؟

ما هي أهمية دراسة الأحياء في المجتمع؟
كيف يمكن أن تؤثر التغيرات البيئية على تنوع الحياة على الأرض؟

ما هي أهمية التكنولوجيا في مجال الطب الحديث؟
كيف يمكن للطب البديل أن يتعامل مع الأمراض التي لا يمكن علاجها بالعلاجات التقليدية؟

[
    'ما هي أهمية علم الفلك في فهم تشكل الكون وتطوره؟\nكيف يمكن استخدام تقنيات الفضاء الحديثة لتحسين معرفتنا بظواهر 
الفلك والنجوم؟',
    'ما هي خصائص الجزيئات في الكيمياء؟\n\nكم عدد أنواع الجزيئات الموجودة في الطبيعة؟',
    'ما هي أهمية تطبيق مبادئ التنمية المستدامة في المجتمعات الحضرية؟ \nكيف يمكن أن يؤثر نمو السكان على الموارد 
الطبيعية والبيئة في المناطق الحضرية؟',
    'ما هي العلاقة بين القوة والجاذبية فيزيائياً؟\n\nهل يمكن أن تؤثر الظروف البيئية على قياس المادة في الفيزياء؟',
    'ما هي العوامل التي تؤثر على توازن المكونات في الخلايا الحية؟\nكيف يتفاعل الجينوم مع البيئة الخارجية لتحقيق 
التوازن البيولوجي؟',
    'ما هي أهمية تعزيز الوعي بالصحة النفسية في المجتمع؟ \nكيف يمكن أن تؤثر التكنولوجيا على صحة النفس البشرية؟',
    'ما هي دورة حياة المنتجات التكنولوجية وتأثيرها على الصناعة؟\n\nكيف يمكن للتقنيات الجديدة مثل الذكاء الاصطناعي 
والروبوتيات أن تؤثر على سوق العمل في المستقبل؟',
    'ما هي أهمية الحفاظ على التنوع البيئي في مواجهة تغير المناخ؟\n\nكيف يمكن للسياسات الاقتصادية أن تؤثر على جودة 
الهواء والتربة في المجتمعات المحلية؟',
    'ما هي أهمية دراسة تأثيرات التغير المناخي على البيئة في الدول النامية؟\n\nهل يمكن أن يؤدي زيادة مستويات الطاقة 
الشمسية إلى تقليل الاعتماد على الوقود الأحفوري؟',
    'ما هي أهمية التعلم الرياضي في الحياة اليومية؟\nكيف يمكن أن تُستخدم الرياضيات في حل المشكلات البيئية؟',
    'ما هي التغيرات التي طرأت على سطح الأرض منذ العصر الحجري؟\n\nكيف تؤثر العمليات الجيولوجية على البيئة المحيطة 
بالمناطق الساحلية؟',
    'ما هي أهمية التوعية بالصحة النفسية في المجتمع؟\nكيف يمكن أن تؤثر العوامل البيئية على الصحة النفسية للفرد؟',
    'ما هي أهمية دراسة الأحياء في المجتمع؟\nكيف يمكن أن تؤثر التغيرات البيئية على تنوع الحياة على الأرض؟',
    'ما هي أهمية التكنولوجيا في مجال الطب الحديث؟\nكيف يمكن للطب البديل أن يتعامل مع الأمراض التي لا يمكن علاجها 
بالعلاجات التقليدية؟'
]

In [19]:
# 3. Responses Generation
RESPONSE_PROMPT_TEMPLATE = """
Given a question, generate a formal positive and loosely related, irrelevant negative response in Arabic.

The questions is: {question}

RESPONSE A should be a formal, positive response that directly answers the question.
RESPONSE B should be a formal, negative response that is loosely related to the question but does not answer it.
Your response should follow this format without any numbers, bullet points, additional text, or the question itself:

RESPONSE A: [Positive response text here]
RESPONSE B: [Negative response text here]
"""
def generate_responses(client, question):
    prompt = RESPONSE_PROMPT_TEMPLATE.format(question=question)
    print('Prompt: ', prompt)
    response = client.chat.completions.create(
        model=modelName,
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content

def response_generator(client, question_list):
    tasks = [generate_responses(client, question) for question in question_list]
    response_list = tasks
    return response_list

question_response_list = response_generator(client, question_list_formatted)
question_response_pair_list = []
for question, response_set in zip(question_list_formatted, question_response_list):
    question_response_pair_list.append(
        {
            "question": question,
            "positive": response_set.split("RESPONSE B:")[0].replace("RESPONSE A:", "").strip(),
            "negative": response_set.split("RESPONSE B:")[-1].split("\n\n")[0].strip()
        }
    )
    


with open(fileName, 'w') as f:
    for item in question_response_pair_list:
        f.write(json.dumps(item))
        f.write('\n')


Prompt:  
Given a question, generate a formal positive and loosely related, irrelevant negative response in Arabic.

The questions is: ما هي أهمية علم الفلك في فهم تشكل الكون وتطوره؟

RESPONSE A should be a formal, positive response that directly answers the question.
RESPONSE B should be a formal, negative response that is loosely related to the question but does not answer it.
Your response should follow this format without any numbers, bullet points, additional text, or the question 
itself:

RESPONSE A: [Positive response text here]
RESPONSE B: [Negative response text here]

RESPONSE A: يعد علم الفلك من العلوم الأساسية التي تساعدنا في فهم تشكل الكون وتطوره، حيث يعمل على دراسة حركة النجوم 
والكواكب والمذنبات وجميع الأجرام السماوية الأخرى، مما يساعدنا في فهم كيفية تكون النظام الشمسي وكيف ساءل عن طريق 
الزمن.

RESPONSE B: تتمتع بعض الثقافات القديمة بالاهتمام الكبير بدراسة حركة النجوم والكواكب، حيث كانوا يعتبرونها علامة على 
التغيرات التي تحدث في العالم المادي، مثل تغير مواسم الزراعة أو حدوث الزلازل.

Prompt:  
Given a question, generate a formal positive and loosely related, irrelevant negative response in Arabic.

The questions is: كيف يمكن استخدام تقنيات الفضاء الحديثة لتحسين معرفتنا بظواهر الفلك والنجوم؟

RESPONSE A should be a formal, positive response that directly answers the question.
RESPONSE B should be a formal, negative response that is loosely related to the question but does not answer it.
Your response should follow this format without any numbers, bullet points, additional text, or the question 
itself:

RESPONSE A: [Positive response text here]
RESPONSE B: [Negative response text here]

RESPONSE A: يمكننا استخدام تقنيات الفضاء الحديثة مثل الأقمار الصناعية والمراصد الفلكية لتحسين معرفتنا بظواهر الفلك 
والنجوم، حيث يمكنها توفير صور عالية الدقة وتحليل البيانات الفلكية بشكل دقيق.

RESPONSE B: من الضروري أن ننظم مجتمعاتنا بشكل أفضل لضمان حصول جميع الأفراد على فرص متساوية في الحصول على التعليم 
والتدريب، مما يؤدي إلى تحسين مستوى المعرفة والوعي بين الناس.

Prompt:  
Given a question, generate a formal positive and loosely related, irrelevant negative response in Arabic.

The questions is: ما هي خصائص الجزيئات في الكيمياء؟

RESPONSE A should be a formal, positive response that directly answers the question.
RESPONSE B should be a formal, negative response that is loosely related to the question but does not answer it.
Your response should follow this format without any numbers, bullet points, additional text, or the question 
itself:

RESPONSE A: [Positive response text here]
RESPONSE B: [Negative response text here]

...

RESPONSE A: يمكن أن تؤثر التغيرات البيئية على تنوع الحياة على الأرض من خلال تغيير ظروف المناخ، مما يؤدي إلى اختفاء 
بعض الأنواع وتحول بيئة أخرى. كما يمكن أن تؤثر على توازن الأحياء وتنويعها، مما يؤدي إلى فقدان التنوع البيولوجي.

RESPONSE B: في الوقت الحالي، هناك حاجة ملحة لتحسين البنية التحتية للطرق السريعة في البلاد، حيث تعاني العديد من 
المناطق من مشاكل في حركة المرور والازدحام. هذا يؤثر على الاقتصاد الوطني ويعيق تنمية الأعمال.

Prompt:  
Given a question, generate a formal positive and loosely related, irrelevant negative response in Arabic.

The questions is: ما هي أهمية التكنولوجيا في مجال الطب الحديث؟

RESPONSE A should be a formal, positive response that directly answers the question.
RESPONSE B should be a formal, negative response that is loosely related to the question but does not answer it.
Your response should follow this format without any numbers, bullet points, additional text, or the question 
itself:

RESPONSE A: [Positive response text here]
RESPONSE B: [Negative response text here]

RESPONSE A: تعتبر التكنولوجيا من أهم العوامل التي تساهم في تقدم الطب الحديث، حيث تتيح للعلماء والباحثين استخدام 
أدوات ووسائل حديثة لتحليل وتطوير الأدوية والمعدات الطبية، مما يؤدي إلى تحسين جودة الرعاية الصحية وتقليل معدلات 
الأمراض.

RESPONSE B: إن التكنولوجيا لها تأثير كبير على الاقتصاد العالمي، حيث تسهم في زيادة الإنتاجية وخفض تكاليف الإنتاج، 
مما يؤدي إلى نمو اقتصادي سريع.

Prompt:  
Given a question, generate a formal positive and loosely related, irrelevant negative response in Arabic.

The questions is: كيف يمكن للطب البديل أن يتعامل مع الأمراض التي لا يمكن علاجها بالعلاجات التقليدية؟

RESPONSE A should be a formal, positive response that directly answers the question.
RESPONSE B should be a formal, negative response that is loosely related to the question but does not answer it.
Your response should follow this format without any numbers, bullet points, additional text, or the question 
itself:

RESPONSE A: [Positive response text here]
RESPONSE B: [Negative response text here]

RESPONSE A: يمكن للطب البديل أن يتعامل مع الأمراض التي لا يمكن علاجها بالعلاجات التقليدية من خلال استخدام تقنيات 
مثل العلاج بالأشعة تحت الحمراء والتدليك والعلاج بالطاقة، مما يساعد في تحسين الصحة العامة وتحفيز الجهاز المناعي.

RESPONSE B: يعتبر تطوير البنية التحتية الصحية في البلدان النامية من الأهمية بمكان لضمان الوصول إلى خدمات الرعاية 
الصحية المتاحة.

In [20]:
scoringModelName = "gemma2:9b-instruct-fp16"

In [21]:
import json
with open(fileName) as f:
    data = [json.loads(line) for line in f]
    
print(data[0])

{
    'question': 'ما هي أهمية علم الفلك في فهم تشكل الكون وتطوره؟',
    'positive': 'يعد علم الفلك من العلوم الأساسية التي تساعدنا في فهم تشكل الكون وتطوره، حيث يعمل على دراسة حركة 
النجوم والكواكب والمذنبات وجميع الأجرام السماوية الأخرى، مما يساعدنا في فهم كيفية تكون النظام الشمسي وكيف ساءل عن 
طريق الزمن.',
    'negative': 'تتمتع بعض الثقافات القديمة بالاهتمام الكبير بدراسة حركة النجوم والكواكب، حيث كانوا يعتبرونها علامة
على التغيرات التي تحدث في العالم المادي، مثل تغير مواسم الزراعة أو حدوث الزلازل.'
}

In [22]:
RESPONSE_PROMPT_TEMPLATE = """
Acting as an Arabic language expert and a subject matter expert on the provided question, evaluate the quality of the following positive answer based on its accuracy, relevance, and clarity in addressing the question. Rate the answer on a confidence scale from 1 to 10, where 1 represents the lowest confidence in the answer's validity and 10 represents the highest. If the question or answer is missing, incomplete, appears incorrect, or contains any words that are neither English nor Arabic, respond with 0. Only respond with the evaluation number.

Question: {question}
Positive Answer: {positive}
"""
def generate_evaluation(client, item):
    question = item['question']
    positive = item['positive']
    prompt = RESPONSE_PROMPT_TEMPLATE.format(question=question, positive=positive)
    print('Prompt: ', prompt)
    response = client.chat.completions.create(
        model=modelName,
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    print(response.choices[0].message.content)
    return int(response.choices[0].message.content)
finalItems = []

for item in data:
    if(generate_evaluation(client, item) > 7):
        finalItems.append(item)

Prompt:  
Acting as an Arabic language expert and a subject matter expert on the provided question, evaluate the quality of 
the following positive answer based on its accuracy, relevance, and clarity in addressing the question. Rate the 
answer on a confidence scale from 1 to 10, where 1 represents the lowest confidence in the answer's validity and 10
represents the highest. If the question or answer is missing, incomplete, appears incorrect, or contains any words 
that are neither English nor Arabic, respond with 0. Only respond with the evaluation number.

Question: ما هي أهمية علم الفلك في فهم تشكل الكون وتطوره؟
Positive Answer: يعد علم الفلك من العلوم الأساسية التي تساعدنا في فهم تشكل الكون وتطوره، حيث يعمل على دراسة حركة 
النجوم والكواكب والمذنبات وجميع الأجرام السماوية الأخرى، مما يساعدنا في فهم كيفية تكون النظام الشمسي وكيف ساءل عن 
طريق الزمن.

10

Prompt:  
Acting as an Arabic language expert and a subject matter expert on the provided question, evaluate the quality of 
the following positive answer based on its accuracy, relevance, and clarity in addressing the question. Rate the 
answer on a confidence scale from 1 to 10, where 1 represents the lowest confidence in the answer's validity and 10
represents the highest. If the question or answer is missing, incomplete, appears incorrect, or contains any words 
that are neither English nor Arabic, respond with 0. Only respond with the evaluation number.

Question: كيف يمكن استخدام تقنيات الفضاء الحديثة لتحسين معرفتنا بظواهر الفلك والنجوم؟
Positive Answer: يمكننا استخدام تقنيات الفضاء الحديثة مثل الأقمار الصناعية والمراصد الفلكية لتحسين معرفتنا بظواهر 
الفلك والنجوم، حيث يمكنها توفير صور عالية الدقة وتحليل البيانات الفلكية بشكل دقيق.

10

Prompt:  
Acting as an Arabic language expert and a subject matter expert on the provided question, evaluate the quality of 
the following positive answer based on its accuracy, relevance, and clarity in addressing the question. Rate the 
answer on a confidence scale from 1 to 10, where 1 represents the lowest confidence in the answer's validity and 10
represents the highest. If the question or answer is missing, incomplete, appears incorrect, or contains any words 
that are neither English nor Arabic, respond with 0. Only respond with the evaluation number.

Question: ما هي خصائص الجزيئات في الكيمياء؟
Positive Answer: تعتبر الجزيئات في الكيمياء هي الوحدات الأساسية التي تتألف منها المواد، وتتميز بخصائصها الفريدة مثل
الشكل والوزن والمادة المكونة لها، وتتحد معًا لتشكيل جزيئات أكبر وتشكيل مادة جديدة.

...

10

Prompt:  
Acting as an Arabic language expert and a subject matter expert on the provided question, evaluate the quality of 
the following positive answer based on its accuracy, relevance, and clarity in addressing the question. Rate the 
answer on a confidence scale from 1 to 10, where 1 represents the lowest confidence in the answer's validity and 10
represents the highest. If the question or answer is missing, incomplete, appears incorrect, or contains any words 
that are neither English nor Arabic, respond with 0. Only respond with the evaluation number.

Question: ما هي أهمية التكنولوجيا في مجال الطب الحديث؟
Positive Answer: تعتبر التكنولوجيا من أهم العوامل التي تساهم في تقدم الطب الحديث، حيث تتيح للعلماء والباحثين 
استخدام أدوات ووسائل حديثة لتحليل وتطوير الأدوية والمعدات الطبية، مما يؤدي إلى تحسين جودة الرعاية الصحية وتقليل 
معدلات الأمراض.

10

Prompt:  
Acting as an Arabic language expert and a subject matter expert on the provided question, evaluate the quality of 
the following positive answer based on its accuracy, relevance, and clarity in addressing the question. Rate the 
answer on a confidence scale from 1 to 10, where 1 represents the lowest confidence in the answer's validity and 10
represents the highest. If the question or answer is missing, incomplete, appears incorrect, or contains any words 
that are neither English nor Arabic, respond with 0. Only respond with the evaluation number.

Question: كيف يمكن للطب البديل أن يتعامل مع الأمراض التي لا يمكن علاجها بالعلاجات التقليدية؟
Positive Answer: يمكن للطب البديل أن يتعامل مع الأمراض التي لا يمكن علاجها بالعلاجات التقليدية من خلال استخدام 
تقنيات مثل العلاج بالأشعة تحت الحمراء والتدليك والعلاج بالطاقة، مما يساعد في تحسين الصحة العامة وتحفيز الجهاز 
المناعي.

9

In [23]:
with open(f"final-{topic}.jsonl".format(topic), 'w') as f:
    for item in finalItems:
        item['topic'] = topic
        f.write(json.dumps(item))
        f.write('\n')